## creating the database. 

In [52]:
# importing required libraries 
import pandas as pd
import numpy as np
import psycopg2 
from sqlalchemy import create_engine 
import os 
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")

### Normal Form analysis:


#### Movie Df 

movie_df is in its first normal form as all the elements in the data frame are atomic. There is no list observed in the data frame. It has one primary key which is the movie id 

movie_df is also in second normal form as no features depend on part of the primary key(I guess there is no such thing as part of the primary key in this case as the primary key is just one column)

movie_df is also in third normal form as no features are transitive depend on another column


#### Genre DF

genre_df is in its first normal form as all the elements in the data frame are atomic. There is no list observed in the data frame. It has one primary key which is movie id 

genre_df is also in second normal form as no features depend on part of the primary key(I guess there is no such thing as part of the primary key in this case as the primary key is just one column)

genre_df is also in third normal form as no features are transitive depend on another column

In [54]:

## building dbserver
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")
dbserver = psycopg2.connect(
    user = 'postgres',
    password = POSTGRES_PASSWORD,
    host = 'postgres',
    port = '5432'
)
dbserver.autocommit = True

# create cursor
cursor = dbserver.cursor()

#construct the database 
try:
    cursor.execute('CREATE DATABASE movie')
except:
    cursor.execute('DROP DATABASE movie')
    cursor.execute('CREATE DATABASE movie')

# create engine
engine = create_engine("postgresql+psycopg2://{user}:{pw}@{service}:{port}/{db}".format(
    user = 'postgres', pw = POSTGRES_PASSWORD, service= "postgres", port=5432,db = 'movie', 
))



In [56]:

# read the csv file into the dataframes.
movies_df = pd.read_csv("movies.csv",lineterminator='\n'
genre_df = pd.read_csv('genres.csv',lineterminator='\n')
cast_member_df = pd.read_csv("cast_members.csv")
cast_lookup_df = pd.read_csv("cast_lookup.csv")
release_date_df = pd.read_csv("movie_release_date.csv")
country_code_df = pd.read_csv("country_code_lookup.csv")
reviews_df = pd.read_csv("reviews.csv")
language_df = pd.read_csv("language.csv")
# creating the database from the pandas dataframes.
movies_df.to_sql('movies',con = engine, index = False, chunksize = 1000, if_exists = 'replace')
genre_df.to_sql('genres',con = engine, index = False, chunksize = 500, if_exists = 'replace')
cast_member_df.to_sql("cast_members",con = engine, index = False, chunksize = 500, if_exists = 'replace')
cast_lookup_df.to_sql("movie_cast",con = engine, index = False, chunksize = 500, if_exists = 'replace')
release_date_df.to_sql("movie_release",con = engine, index = False, chunksize = 500, if_exists = 'replace')
country_code_df.to_sql("country_code",con = engine, index = False, chunksize = 500, if_exists = 'replace')
reviews_df.to_sql("reviews",con = engine, index = False, chunksize = 500, if_exists = 'replace')
language_df.to_sql("language",con = engine, index = False, chunksize = 500, if_exists = 'replace')




In [16]:
### Just to double check if the data base is correctly build, I'll do some query on the database
myquery = '''
SELECT * 
FROM movies
'''
pd.read_sql_query(myquery, con = engine)


,id,title,release_date,language,description,runtime,poster_image,score
0,1075794,Leo,2023-11-17,en,Jaded 74-year-old lizard Leo has been stuck in...,102,https://image.tmdb.org/t/p/original/pD6sL4vntU...,7.9
1,901362,Trolls Band Together,2023-10-12,en,"When Branch’s brother, Floyd, is kidnapped for...",92,https://image.tmdb.org/t/p/original/sEaLO9s7CI...,7.2
2,872585,Oppenheimer,2023-07-19,en,The story of J. Robert Oppenheimer's role in t...,181,https://image.tmdb.org/t/p/original/8Gxv8gSFCU...,8.2
3,670292,The Creator,2023-09-27,en,Amid a future war between the human race and t...,134,https://image.tmdb.org/t/p/original/vBZ0qvaRxq...,7.1
4,507089,Five Nights at Freddy's,2023-10-25,en,"Recently fired and desperate for work, a troub...",110,https://image.tmdb.org/t/p/original/j9mH1pr3Ia...,7.9
...,...,...,...,...,...,...,...,...
10036,6471,The Jerk,1979-12-14,en,After discovering he's not really black like t...,94,https://image.tmdb.org/t/p/original/vKfBXwcmnd...,6.9
10037,13350,Scooby-Doo and the Ghoul School,1988-10-16,en,"Scooby, Shaggy and Scrappy Doo are on their wa...",92,https://image.tmdb.org/t/p/original/duMdoHLQvn...,7.4
10038,1112202,ആന്റണി,2023-12-01,ml,Antony is a ruthless gangster in a town and ha...,135,https://image.tmdb.org/t/p/original/8FvGirK0L6...,0.0
10039,714651,소년들,2023-11-01,ko,"A story about an investigating team leader, Hw...",124,https://image.tmdb.org/t/p/original/c8Hslxtlkj...,7.0


In [22]:
##filter by movie runtime and score
myquery = '''
SELECT * 
FROM movies
WHERE runtime >120 AND score>8
'''
pd.read_sql_query(myquery, con = engine)

,id,title,release_date,language,description,runtime,poster_image,score
0,872585,Oppenheimer,2023-07-19,en,The story of J. Robert Oppenheimer's role in t...,181,https://image.tmdb.org/t/p/original/8Gxv8gSFCU...,8.2
1,569094,Spider-Man: Across the Spider-Verse,2023-05-31,en,"After reuniting with Gwen Stacy, Brooklyn’s fu...",140,https://image.tmdb.org/t/p/original/8Vt6mWEReu...,8.4
2,678512,Sound of Freedom,2023-07-03,en,"The story of Tim Ballard, a former US governme...",131,https://image.tmdb.org/t/p/original/qA5kPYZA7F...,8.1
3,299536,Avengers: Infinity War,2018-04-25,en,As the Avengers and their allies have continue...,149,https://image.tmdb.org/t/p/original/7WsyChQLEf...,8.3
4,361743,Top Gun: Maverick,2022-05-24,en,After more than thirty years of service as one...,131,https://image.tmdb.org/t/p/original/62HCnUTziy...,8.2
...,...,...,...,...,...,...,...,...
125,76115,The Phantom of the Opera at the Royal Albert Hall,2011-09-27,en,"A disfigured musical genius, hidden away in th...",160,https://image.tmdb.org/t/p/original/1vf1ryVPBS...,8.5
126,26022,My Name Is Khan,2010-02-10,hi,"Rizwan Khan, a Muslim from the Borivali sectio...",165,https://image.tmdb.org/t/p/original/5Y36lCiNyy...,8.1
127,630566,Clouds,2020-10-09,en,Young musician Zach Sobiech discovers his canc...,121,https://image.tmdb.org/t/p/original/d0OdD1I8qA...,8.3
128,16672,砂の女,1964-02-15,ja,An entomologist suffers extreme psychological ...,147,https://image.tmdb.org/t/p/original/f0JpsMQ9oE...,8.3


In [24]:
##filter by language if you want Japanese movies
myquery = '''
SELECT * 
FROM movies
WHERE language = 'ja'
'''
pd.read_sql_query(myquery, con = engine)

,id,title,release_date,language,description,runtime,poster_image,score
0,116776,ドラゴンボール 魔訶不思議大冒険,1988-07-09,ja,Master Roshi has succeeded at the one mission ...,48,https://image.tmdb.org/t/p/original/5aXG0B3TYT...,6.8
1,545742,ワンピース エピソード オブ 空島,2018-08-25,ja,"One day, a giant ship falls onto the Straw Hat...",104,https://image.tmdb.org/t/p/original/whvgyEpPqv...,7.2
2,900667,ONE PIECE FILM RED,2022-08-06,ja,Uta — the most beloved singer in the world. He...,115,https://image.tmdb.org/t/p/original/ogDXuVkO92...,7.3
3,1047041,名探偵コナン 黒鉄の魚影（サブマリン）,2023-04-14,ja,The 26th movie entry in the “Detective Conan” ...,110,https://image.tmdb.org/t/p/original/ksQ8uNgoWs...,6.7
4,1178734,ナックルガール,2023-10-25,ja,Ran participates in a death match in order to ...,107,https://image.tmdb.org/t/p/original/3mQ8tK7xpx...,6.8
...,...,...,...,...,...,...,...,...
573,185648,団鬼六 少女木馬責め,1982-12-03,ja,Nami is a masochistic high school student who ...,68,https://image.tmdb.org/t/p/original/AhtNrNZ3Gk...,6.2
574,80788,団鬼六 美教師地獄責め,1985-12-14,ja,Roman Porno from 1985. A young teacher heads o...,67,https://image.tmdb.org/t/p/original/gMyMOsF1vo...,5.7
575,1178734,ナックルガール,2023-10-25,ja,Ran participates in a death match in order to ...,107,https://image.tmdb.org/t/p/original/3mQ8tK7xpx...,6.5
576,976893,Perfect Days,2023-11-10,ja,Hirayama seems utterly content with his simple...,123,https://image.tmdb.org/t/p/original/dENgZn7CD6...,4.9


In [36]:
# merging the movie with genre
myquery = '''
SELECT m.id,
        m.title,
        m.score,
        g.genre
FROM movies m
INNER JOIN genres g
    ON m.id = g.id
WHERE language = 'ja'
'''
pd.read_sql_query(myquery, con = engine)

,id,title,score,genre
0,116776,ドラゴンボール 魔訶不思議大冒険,6.8,Action
1,116776,ドラゴンボール 魔訶不思議大冒険,6.8,Animation
2,545742,ワンピース エピソード オブ 空島,7.2,Animation
3,545742,ワンピース エピソード オブ 空島,7.2,Action
4,545742,ワンピース エピソード オブ 空島,7.2,Adventure
...,...,...,...,...
1898,347183,ハイキュー!! 終わりと始まり,8.5,Animation
1899,347183,ハイキュー!! 終わりと始まり,8.5,Comedy
1900,347183,ハイキュー!! 終わりと始まり,8.5,Drama
1901,1056803,青春ブタ野郎はおでかけシスターの夢を見ない,8.8,Animation


In [79]:
review = pd.read_csv("reviews.csv",lineterminator='\n')
review.dtypes


movie_id         int64
author_name     object
creat_date      object
rating         float64
content         object
dtype: object